# REPORT 5 - Switching current vaccines to variant adapted vaccines

In this notebook we analyse the public health impact achieved through variant adapted vaccines versus current vaccines in populations with different levels of infection- versus vaccine-induced seroprevalance. We use the `warwickmodel`, built by Universities of Warwick and Lancaster, with population data from 8 countries with very different socio-economic profiles. We assume an initial boosting campaign in the population, with no subsequent boosters being deployed during the simulation. 

The infection dynamics are run for:
 - Dates: **15 Feb 2020** - **25 June 2021**;
 - Countries of interest: **United Kingdom**, **France**;
 - Number of boosters deployed: **10%** of the population.

*The Warwick model is built by Universities of Warwick and Lancaster.*

In [1]:
# Load necessary libraries
import os
import copy
import numpy as np
import pandas as pd
import scipy
import epimodels as em
import warwickmodel as wm
import matplotlib
import plotly.graph_objects as go
from matplotlib import pyplot as plt
from iteration_utilities import deepflatten

## Model Setup
### Define setup matrices for the WarwickLanc Model

In [2]:
# Populate the model
total_days =  150
regions = ['United Kingdom', 'Canada', 'Brazil', 'South Africa', 'Kenya', 'Philippines', 'Sierra Leone', 'Syria']
age_groups = ['0-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39',
              '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75+']

regimes = np.arange(1, 101, 20).tolist()

# Add folder path to data file
path = os.path.join('../data/')

### Read in corresponding data files for the countries considered for all 100 regimes

In [3]:
# Matrices contact
matrices_contact = []
matrices_region = []
time_changes_contact = []
time_changes_region = []

# Vaccine effects
nu_tra = []
nu_symp = []
nu_inf = []
nu_sev_h = []
nu_sev_d = []

# Parameters
omega = []
alpha = []
gamma = []
tau = []
we = []

# Initial conditions
susceptibles_IC = []
exposed1_IC = []
exposed2_IC = []
exposed3_IC = []
exposed4_IC = []
exposed5_IC = []
infectives_sym_IC = []
infectives_asym_IC = []
recovered_IC = []

# Risk factors
d = []
beta = []

# Probabilities of proceeding to severe outcomes
pItoH = []
pHtoD = []

# Distribution of delays before proceeding to severe outcomes
dItoH = []
dHtoD = []

for R in regimes:
        regimes_matrices_region = []

        # Initial state of the system
        weeks_matrices_region = []
        for r in regions:
                region_data_matrix = pd.read_csv(
                        os.path.join(path, '{}/Contacts_{}.csv'.format(r, R)),
                        header=None, dtype=np.float64)
                regional = em.RegionMatrix(r, age_groups, region_data_matrix)
                weeks_matrices_region.append(regional)

        regimes_matrices_region.append(weeks_matrices_region)

        contacts = em.ContactMatrix(age_groups, np.ones((len(age_groups), len(age_groups))))
        regimes_matrices_contact = [contacts]

        matrices_region.append(regimes_matrices_region)
        matrices_contact.append(regimes_matrices_contact)

        # Matrices contact
        time_changes_contact.append([1])
        time_changes_region.append([1])

        # Over 75 population fractions
        frac_pop_over75 = []

        for r in regions:
                IC_df = pd.read_csv(
                        os.path.join(path, '{}/Start_pop_{}.csv'.format(r, R)),
                        skiprows=15,
                        header=None, dtype=np.float64)

                frac_pop_over75.append((1/np.sum(np.asarray(IC_df))) * np.sum(np.asarray(IC_df),axis=1))

        # Risk Factors
        extended_regimes_d = []
        regimes_d = []
        regimes_beta = []

        for r, reg in enumerate(regions):
                RF_df = pd.read_csv(
                        os.path.join(path, '{}/Risks_{}.csv'.format(reg, R)),
                        dtype=np.float64)
                extended_d = RF_df['symptom_risk'].tolist()
                extended_beta = RF_df['susceptibility'].tolist()

                extended_regimes_d.append(extended_d)

                regimes_d.append(extended_d[:15] + [np.sum(np.multiply(extended_d[15:], frac_pop_over75[r]))])
                regimes_beta.append(extended_beta[:15] + [np.sum(np.multiply(extended_beta[15:], frac_pop_over75[r]))])

        d.append(regimes_d)
        beta.append(regimes_beta)

        # Vaccine effects
        eff_df = pd.read_csv(
                os.path.join(path, 'global_parameters/efficacies_{}.csv'.format(R)),
                usecols=range(1,5), dtype=np.float64)
        
        VE_i = eff_df['Infection_eff']
        VE_s = eff_df['Symptom_eff']
        VE_h = eff_df['Hosp_eff']
        VE_d = eff_df['Death_eff']

        VE_d = np.divide(VE_d-VE_h, 1-VE_h)
        VE_h = np.divide(VE_h-VE_i, 1-VE_i)
        VE_s = np.divide(VE_s-VE_i, 1-VE_i)

        regimes_nu_tra = [1] * 6
        regimes_nu_symp = np.nan_to_num(1 - VE_s).tolist()
        regimes_nu_inf = np.nan_to_num(1 - VE_i).tolist()
        regimes_nu_sev_h = np.nan_to_num(1 - VE_h).tolist()
        regimes_nu_sev_d = np.nan_to_num(1 - VE_d).tolist()

        nu_tra.append(regimes_nu_tra)
        nu_symp.append(regimes_nu_symp)
        nu_inf.append(regimes_nu_inf)
        nu_sev_h.append(regimes_nu_sev_h)
        nu_sev_d.append(regimes_nu_sev_d)

        # Parameters
        param_df = pd.read_csv(
                os.path.join(path, 'global_parameters/parameters_{}.csv'.format(R)),
                dtype=np.float64)

        regimes_omega = param_df['transmission'].tolist()[0]
        regimes_alpha = 1
        regimes_gamma = param_df['recovery'].tolist()[0]
        regimes_tau = param_df['asymptomatic_transmission'].tolist()[0]
        regimes_we = [param_df['waning_rate'].tolist()[0]] * 2 + [0]

        omega.append(regimes_omega)
        alpha.append(regimes_alpha)
        gamma.append(regimes_gamma)
        tau.append(regimes_tau)
        we.append(regimes_we)

        # Initial conditions
        regimes_susceptibles_IC = []
        regimes_exposed1_IC = []
        regimes_exposed2_IC = []
        regimes_exposed3_IC = []
        regimes_exposed4_IC = []
        regimes_exposed5_IC = []
        regimes_infectives_sym_IC = []
        regimes_infectives_asym_IC = []
        regimes_recovered_IC = []

        # Susceptible
        for r in regions:
                IC_df = pd.read_csv(
                        os.path.join(path, '{}/Start_pop_{}.csv'.format(r, R)),
                        usecols=range(0, 5),
                        header=None, dtype=np.float64)

                extended_S = np.asarray(IC_df)
                under_75_S = extended_S[:15, :]
                over_75_S = extended_S[15:, :]
                reduced_S = np.vstack((under_75_S, np.sum(over_75_S, axis=0)))
                regimes_susceptibles_IC.append(
                        reduced_S.flatten('F').tolist() + [0] * len(age_groups))

        # Exposed 1
        for r in regions:
                IC_df = pd.read_csv(
                        os.path.join(path, '{}/Start_pop_{}.csv'.format(r, R)),
                        usecols=range(5, 10),
                        header=None, dtype=np.float64)

                extended_E1 = np.asarray(IC_df)
                under_75_E1 = extended_E1[:15, :]
                over_75_E1 = extended_E1[15:, :]
                reduced_E1 = np.vstack((under_75_E1, np.sum(over_75_E1, axis=0)))
                regimes_exposed1_IC.append(
                        reduced_E1.flatten('F').tolist() + [0] * len(age_groups))

        # Exposed 2
        for r in regions:
                IC_df = pd.read_csv(
                        os.path.join(path, '{}/Start_pop_{}.csv'.format(r, R)),
                        usecols=range(10, 15),
                        header=None, dtype=np.float64)

                extended_E2 = np.asarray(IC_df)
                under_75_E2 = extended_E2[:15, :]
                over_75_E2 = extended_E2[15:, :]
                reduced_E2 = np.vstack((under_75_E2, np.sum(over_75_E2, axis=0)))
                regimes_exposed2_IC.append(
                        reduced_E2.flatten('F').tolist() + [0] * len(age_groups))

        # Exposed 3
        for r in regions:
                IC_df = pd.read_csv(
                        os.path.join(path, '{}/Start_pop_{}.csv'.format(r, R)),
                        usecols=range(15, 20),
                        header=None, dtype=np.float64)

                extended_E3 = np.asarray(IC_df)
                under_75_E3 = extended_E3[:15, :]
                over_75_E3 = extended_E3[15:, :]
                reduced_E3 = np.vstack((under_75_E3, np.sum(over_75_E3, axis=0)))
                regimes_exposed3_IC.append(
                        reduced_E3.flatten('F').tolist() + [0] * len(age_groups))

        # Exposed 4
        for r in regions:
                IC_df = pd.read_csv(
                        os.path.join(path, '{}/Start_pop_{}.csv'.format(r, R)),
                        usecols=range(20, 25),
                        header=None, dtype=np.float64)

                extended_E4 = np.asarray(IC_df)
                under_75_E4 = extended_E4[:15, :]
                over_75_E4 = extended_E4[15:, :]
                reduced_E4 = np.vstack((under_75_E4, np.sum(over_75_E4, axis=0)))
                regimes_exposed4_IC.append(
                        reduced_E4.flatten('F').tolist() + [0] * len(age_groups))

        # Exposed 5
        for r in regions:
                IC_df = pd.read_csv(
                        os.path.join(path, '{}/Start_pop_{}.csv'.format(r, R)),
                        usecols=range(25, 30),
                        header=None, dtype=np.float64)

                extended_E5 = np.asarray(IC_df)
                under_75_E5 = extended_E5[:15, :]
                over_75_E5 = extended_E5[15:, :]
                reduced_E5 = np.vstack((under_75_E5, np.sum(over_75_E5, axis=0)))
                regimes_exposed5_IC.append(
                        reduced_E5.flatten('F').tolist() + [0] * len(age_groups))

        # Symptomatic & Asymptomatic Infectious
        for _, r in enumerate(regions):
                IC_df = pd.read_csv(
                        os.path.join(path, '{}/Start_pop_{}.csv'.format(r, R)),
                        usecols=range(30, 35),
                        header=None, dtype=np.float64)

                extended_I = np.zeros_like(np.asarray(IC_df))
                extended_I[:, 0] = np.matmul(np.diag(regimes_nu_symp[0] * np.array(extended_regimes_d[_])), np.asarray(IC_df)[:, 0])
                extended_I[:, 1] = np.matmul(np.diag(regimes_nu_symp[1] * np.array(extended_regimes_d[_])), np.asarray(IC_df)[:, 1])
                extended_I[:, 2] = np.matmul(np.diag(regimes_nu_symp[2] * np.array(extended_regimes_d[_])), np.asarray(IC_df)[:, 2])
                extended_I[:, 3] = np.matmul(np.diag(regimes_nu_symp[3] * np.array(extended_regimes_d[_])), np.asarray(IC_df)[:, 3])
                extended_I[:, 4] = np.matmul(np.diag(regimes_nu_symp[4] * np.array(extended_regimes_d[_])), np.asarray(IC_df)[:, 4])
                under_75_I = extended_I[:15, :]
                over_75_I = extended_I[15:, :]
                reduced_I = np.vstack((under_75_I, np.sum(over_75_I, axis=0)))
                regimes_infectives_sym_IC.append(
                        reduced_I.flatten('F').tolist() + [0] * len(age_groups))

                extended_A = np.zeros_like(np.asarray(IC_df))
                extended_A[:, 0] = np.matmul(np.diag((1 - regimes_nu_symp[0] * np.array(extended_regimes_d[_]))), np.asarray(IC_df)[:, 0])
                extended_A[:, 1] = np.matmul(np.diag((1 - regimes_nu_symp[1] * np.array(extended_regimes_d[_]))), np.asarray(IC_df)[:, 1])
                extended_A[:, 2] = np.matmul(np.diag((1 - regimes_nu_symp[2] * np.array(extended_regimes_d[_]))), np.asarray(IC_df)[:, 2])
                extended_A[:, 3] = np.matmul(np.diag((1 - regimes_nu_symp[3] * np.array(extended_regimes_d[_]))), np.asarray(IC_df)[:, 3])
                extended_A[:, 4] = np.matmul(np.diag((1 - regimes_nu_symp[4] * np.array(extended_regimes_d[_]))), np.asarray(IC_df)[:, 4])
                under_75_A = extended_A[:15, :]
                over_75_A = extended_A[15:, :]
                reduced_A = np.vstack((under_75_A, np.sum(over_75_A, axis=0)))
                regimes_infectives_asym_IC.append(
                        reduced_A.flatten('F').tolist() + [0] * len(age_groups))

        # Recovered
        for r in regions:
                IC_df = pd.read_csv(
                        os.path.join(path, '{}/Start_pop_{}.csv'.format(r, R)),
                        usecols=[35],
                        header=None, dtype=np.float64)

                extended_R = np.asarray(IC_df)
                under_75_R = extended_R[:15, :]
                over_75_R = extended_R[15:, :]
                reduced_R = np.vstack((under_75_R, np.sum(over_75_R, axis=0)))
                regimes_recovered_IC.append(
                        reduced_R.flatten('F').tolist())

        susceptibles_IC.append(regimes_susceptibles_IC)
        exposed1_IC.append(regimes_exposed1_IC)
        exposed2_IC.append(regimes_exposed2_IC)
        exposed3_IC.append(regimes_exposed3_IC)
        exposed4_IC.append(regimes_exposed4_IC)
        exposed5_IC.append(regimes_exposed5_IC)
        infectives_sym_IC.append(regimes_infectives_sym_IC)
        infectives_asym_IC.append(regimes_infectives_asym_IC)
        recovered_IC.append(regimes_recovered_IC)

        # Set time-to-hospitalisation using a Gamma distribution using the mean and standard deviation 
        th_mean = param_df['hosp_lag'].tolist()[0]+0.00001
        th_var = 12.1**2
        theta = th_var / th_mean
        k = th_mean / theta
        time_to_hosp = scipy.stats.gamma(k, scale=theta).pdf(np.arange(1, 31)).tolist()

        # Set time-to-death using a Gamma distribution using the mean and standard deviation
        td_mean = param_df['death_lag'].tolist()[0]
        td_var = 12.1**2
        theta = td_var / td_mean
        k = td_mean / theta
        time_to_death = scipy.stats.gamma(k, scale=theta).pdf(np.arange(1, 31)).tolist()

        # Probabilities of proceeding to severe outcomes
        # Infected -> Hospital
        extended_pItoH = RF_df['hospitalisation_risk'].tolist()

        regimes_pItoH = []
        for r, reg in enumerate(regions):
                regimes_pItoH.append(extended_pItoH[:15] + [np.sum(np.multiply(extended_pItoH[15:], frac_pop_over75[r]))])

        pItoH.append(regimes_pItoH)

        # Hospital -> Death
        extended_pHtoD = RF_df['death_risk'].tolist()

        regimes_pHtoD = []
        for r, reg in enumerate(regions):
                regimes_pHtoD.append(extended_pHtoD[:15] + [np.sum(np.multiply(extended_pHtoD[15:], frac_pop_over75[r]))])

        pHtoD.append(regimes_pHtoD)

        # Distribution of delays before proceeding to severe outcomes
        # Infected -> Hospital
        dItoH.append(time_to_hosp)
        # Hospital -> Death
        dHtoD.append(time_to_death)

# Other parameters
vac=0
vacb=0

adult = np.ones(len(age_groups))
adult[0] = 0
adult[1] = 0
adult[2] = 0

### Calculate total population for each regime

In [4]:
# Compute the number of boosters for each region
total_pop = []

for R in regimes:
        regimes_total_pop = []
        for r in regions:
                IC_df = pd.read_csv(
                        os.path.join(path, '{}/Start_pop_{}.csv'.format(r, R)),
                        header=None, dtype=np.float64)

                regimes_total_pop.append(np.sum(np.asarray(IC_df)))
        total_pop.append(regimes_total_pop)

## Seroprevalance scenarios

Due to inconsistencies in case data collection, it is not unreasonable to assume more people might have been infected or have some sort of advanced protection due to vaccines.

In the case of increased infection-induced seroprevalance, we move people into the R compartment from those in which we assume no or little protection (S and Sw2).
In the case of increased vaccine-induced seroprevalance, we move people into the Sf compartment from those in which we assume no or little protection (S and Sw2).

In [5]:
# Names of seroprevalance scenarios
seroprevalance_scenarios = []

ss_susceptibles_IC = []
ss_recovered_IC = []

**Scenario 1**: Current Initial Conditions

In [6]:
seroprevalance_scenarios.append('Original estimates')

ss_susceptibles_IC.append(susceptibles_IC)
ss_recovered_IC.append(recovered_IC)

**Scenario 2**: 20% Decrease in Existing Immunity

In [7]:
seroprevalance_scenarios.append('Decreased immunity')

increase = 0.3
regimes_new_susceptibles_IC = []
regimes_new_recovered_IC = []

for R, _ in enumerate(regimes):
    new_susceptibles_IC = np.array([
        np.array(susceptibles_IC[R][r].copy()).reshape((6, 16)).transpose() for r in range(len(regions))])
    new_recovered_IC = np.array(recovered_IC[R].copy())

    reg_new_susceptibles_IC = []
    reg_new_recovered_IC = []

    for r, reg in enumerate(regions):
        susceptible_matrix = new_susceptibles_IC[r].copy()
        recovered_matrix = new_recovered_IC[r].copy()

        # Remove extra infection-immune from Sf
        susceptible_matrix[:, 1] *= 1 - increase
        # Remove extra infection-immune from Sb
        susceptible_matrix[:, 2] *= 1 - increase
        # Remove extra infection-immune from Sw1
        susceptible_matrix[:, 3] *= 1 - increase
        # Remove extra infection-immune from Sw2
        susceptible_matrix[:, 4] *= 1 - increase
        # Remove extra infection-immune from R
        recovered_matrix *= 1 - increase

        # Add extra infection-immune from Sw2 to S
        susceptible_matrix[:, 0] += increase * new_susceptibles_IC[r, :, 4]
        # Add extra infection-immune from Sb and R to Sw1
        susceptible_matrix[:, 3] += increase * (new_susceptibles_IC[r, :, 2] + new_recovered_IC[r, :])
        # Add extra infection-immune from Sf and Sw1 to Sw2
        susceptible_matrix[:, 4] += increase * (new_susceptibles_IC[r, :, 1] + new_susceptibles_IC[r, :, 3])

        reg_new_susceptibles_IC.append(list(deepflatten(susceptible_matrix.transpose())))
        reg_new_recovered_IC.append(list(deepflatten(recovered_matrix)))

    regimes_new_susceptibles_IC.append(reg_new_susceptibles_IC)
    regimes_new_recovered_IC.append(reg_new_recovered_IC)

ss_susceptibles_IC.append(regimes_new_susceptibles_IC)
ss_recovered_IC.append(regimes_new_recovered_IC)

**Scenario 3**: 20% Increase in Existing Immunity

In [8]:
seroprevalance_scenarios.append('Increased immunity')

increase = 0.3
regimes_new_susceptibles_IC = []
regimes_new_recovered_IC = []

for R, _ in enumerate(regimes):
    new_susceptibles_IC = np.array([
        np.array(susceptibles_IC[R][r].copy()).reshape((6, 16)).transpose() for r in range(len(regions))])
    new_recovered_IC = np.array(recovered_IC[R].copy())

    reg_new_susceptibles_IC = []
    reg_new_recovered_IC = []

    for r, reg in enumerate(regions):
        susceptible_matrix = new_susceptibles_IC[r].copy()
        recovered_matrix = new_recovered_IC[r].copy()

        prop_w1 = np.divide(recovered_matrix, recovered_matrix + susceptible_matrix[:, 2])
        prop_w2 = np.divide(susceptible_matrix[:, 1], susceptible_matrix[:, 1] + susceptible_matrix[:, 3])

        # Remove extra infection-immune from S
        susceptible_matrix[:, 0] *= 1 - increase
        # Remove extra infection-immune from Sf
        susceptible_matrix[:, 1] *= 1 - increase
        # Remove extra infection-immune from Sw1
        susceptible_matrix[:, 3] *= 1 - increase
        # Remove extra infection-immune from Sw2
        susceptible_matrix[:, 4] *= 1 - increase

        # Add extra infection-immune from Sw2 to Sf
        susceptible_matrix[:, 1] += increase * np.multiply(prop_w2, new_susceptibles_IC[r, :, 4])
        # Add extra infection-immune from Sf and Sw1 to Sb
        susceptible_matrix[:, 2] += increase * (new_susceptibles_IC[r, :, 1] + np.multiply(1 - prop_w1, new_susceptibles_IC[r, :, 3]))
        # Add extra infection-immune from Sw2 to Sw1
        susceptible_matrix[:, 3] += increase * np.multiply(1-prop_w2, new_susceptibles_IC[r, :, 4])
        # Add extra infection-immune from S to Sw2
        susceptible_matrix[:, 4] += increase * new_susceptibles_IC[r, :, 0]
        # Add extra infection-immune from Sw1 to R
        recovered_matrix += increase * np.multiply(prop_w1, new_susceptibles_IC[r, :, 3])

        reg_new_susceptibles_IC.append(list(deepflatten(susceptible_matrix.transpose())))
        reg_new_recovered_IC.append(list(deepflatten(recovered_matrix)))

    regimes_new_susceptibles_IC.append(reg_new_susceptibles_IC)
    regimes_new_recovered_IC.append(reg_new_recovered_IC)

ss_susceptibles_IC.append(regimes_new_susceptibles_IC)
ss_recovered_IC.append(regimes_new_recovered_IC)

## Boosting campaign scenarios

We only boost all the susceptibles types, but the recently boosted, and recovered. However those in the R compartment who receive the booster do not move out of the compartment (they have higher immunity than the boosted)

In [9]:
# Maximum percentage booster uptake of each age group
boost_age_percent = 0.9

# Compute the maximum number of boosters we can deploy for each age group in each region
max_boosters = []
max_boosters_for_R = []
max_boosters_for_S = []
max_boosters_for_Sf = []
max_boosters_for_Sw1 = []
max_boosters_for_Sw2 = []

old_boosted = []

for ss, sero_scen in enumerate(seroprevalance_scenarios):
        ss_max_boosters = []
        ss_max_boosters_for_R = []
        ss_max_boosters_for_S = []
        ss_max_boosters_for_Sf = []
        ss_max_boosters_for_Sw1 = []
        ss_max_boosters_for_Sw2 = []

        ss_old_boosted = []

        for R, _ in enumerate(regimes):
                regimes_max_boosters = []
                regimes_max_boosters_for_R = []
                regimes_max_boosters_for_S = []
                regimes_max_boosters_for_Sf = []
                regimes_max_boosters_for_Sw1 = []
                regimes_max_boosters_for_Sw2 = []

                regimes_old_boosted = []

                for r, reg in enumerate(regions):
                        boosters_for_R = boost_age_percent * np.asarray(ss_recovered_IC[ss][R][r])
                        boosters_for_S = boost_age_percent * np.asarray(ss_susceptibles_IC[ss][R][r])[:len(age_groups)]
                        boosters_for_Sf = boost_age_percent * np.asarray(ss_susceptibles_IC[ss][R][r])[len(age_groups):(2*len(age_groups))]
                        boosters_for_Sw1 = boost_age_percent * np.asarray(ss_susceptibles_IC[ss][R][r])[(3*len(age_groups)):(4*len(age_groups))]
                        boosters_for_Sw2 = boost_age_percent * np.asarray(ss_susceptibles_IC[ss][R][r])[(4*len(age_groups)):(5*len(age_groups))]

                        boosted = np.asarray(ss_susceptibles_IC[ss][R][r])[(2*len(age_groups)):(3*len(age_groups))]

                        regimes_max_boosters_for_R.append(boosters_for_R)
                        regimes_max_boosters_for_S.append(boosters_for_S)
                        regimes_max_boosters_for_Sf.append(boosters_for_Sf)
                        regimes_max_boosters_for_Sw1.append(boosters_for_Sw1)
                        regimes_max_boosters_for_Sw2.append(boosters_for_Sw2)

                        regimes_max_boosters.append(
                                boosters_for_R + boosters_for_S + boosters_for_Sf +
                                boosters_for_Sw1 + boosters_for_Sw2)

                        regimes_old_boosted.append(boosted)

                ss_max_boosters_for_R.append(regimes_max_boosters_for_R)
                ss_max_boosters_for_S.append(regimes_max_boosters_for_S)
                ss_max_boosters_for_Sf.append(regimes_max_boosters_for_Sf)
                ss_max_boosters_for_Sw1.append(regimes_max_boosters_for_Sw1)
                ss_max_boosters_for_Sw2.append(regimes_max_boosters_for_Sw2)
                ss_max_boosters.append(regimes_max_boosters)

                ss_old_boosted.append(regimes_old_boosted)

        max_boosters_for_R.append(ss_max_boosters_for_R)
        max_boosters_for_S.append(ss_max_boosters_for_S)
        max_boosters_for_Sf.append(ss_max_boosters_for_Sf)
        max_boosters_for_Sw1.append(ss_max_boosters_for_Sw1)
        max_boosters_for_Sw2.append(ss_max_boosters_for_Sw2)
        max_boosters.append(ss_max_boosters)

        old_boosted.append(ss_old_boosted)

# Create list of new susceptible_ICs for each vaccination scenario
scenario_susceptibles_IC = []
scenario_new_boosted = []
scenario_boost_pop_percent = []
scenario_names = []
age_boosting_scenario_order = []

**Scenario 1**: Prioritising those 75+

In [10]:
scenario_names.append('Prioritise 75+')
age_boosting_scenario_order.append((np.arange(16, 0, -1) - 1).tolist())
scenario_boost_pop_percent.append(0.1)

**Scenario 2**: Prioritising those 60-74 then 75+

In [11]:
scenario_names.append('Prioritise 60-74 then 75+')
age_boosting_scenario_order.append([range(12, 15), 15] + (np.arange(12, 0, -1) - 1).tolist())
scenario_boost_pop_percent.append(0.1)

**Scenario 3**: Prioritising those 60-74 then 50-59, then 75+

In [12]:
scenario_names.append('Prioritise those 60-74 then 50-59, then 75+')
age_boosting_scenario_order.append([range(12, 15), range(10, 12), 15] + (np.arange(10, 0, -1) - 1).tolist())
scenario_boost_pop_percent.append(0.1)

**Scenario 4**: Prioritising those 50-59, then 60-74 then 75+

In [13]:
scenario_names.append('Prioritise those 50-59 then 60-74, then 75+')
age_boosting_scenario_order.append([range(10, 12), range(12, 15), 15] + (np.arange(10, 0, -1) - 1).tolist())
scenario_boost_pop_percent.append(0.1)

**Scenario 5**: Prioritising those 20-49, then 50-74 then 75+

In [14]:
scenario_names.append('Prioritise those 20-49, then 50-74 then 75+')
age_boosting_scenario_order.append([range(4, 10), range(10, 15), 15] + (np.arange(4, 0, -1) - 1).tolist())
scenario_boost_pop_percent.append(0.1)

**Scenario 6**: Prioritising those 20-49, then 75+, then 50-74

In [15]:
scenario_names.append('Prioritise those 20-49, then 75+, then 50-74')
age_boosting_scenario_order.append([range(4, 10), 15, range(10, 15)] + (np.arange(4, 0, -1) - 1).tolist())
scenario_boost_pop_percent.append(0.1)

**Scenario 7**: No boosters

In [16]:
scenario_names.append('No boosters')
age_boosting_scenario_order.append((np.arange(16, 0, -1) - 1).tolist())
scenario_boost_pop_percent.append(0)

### Compute new susceptible conditions based on each boosting scenario
We use the Sw3 compartment to house the displaced individuals

In [17]:
for ss, sero_scen in enumerate(seroprevalance_scenarios):
    scenario_regimes_new_susceptibles_IC = []
    scenario_regimes_new_boosted = []
    for s, _ in enumerate(scenario_names):
        regimes_new_susceptibles_IC = []
        regimes_new_boosted = []
        scenario_new_boosters = (scenario_boost_pop_percent[s] * np.array(total_pop)).tolist()

        for R, _ in enumerate(regimes):
            new_susceptibles_IC = [
                np.array(ss_susceptibles_IC[ss][R][r]).reshape((6, 16)).transpose() for r in range(len(regions))]
            
            new_susceptibles_IC = np.array(new_susceptibles_IC)
            regimes_scenario_new_boosters = scenario_new_boosters[R]
            reg_new_susceptibles_IC = []

            boosted = np.zeros_like(new_susceptibles_IC[:, :, 2])

            for r, reg in enumerate(regions):
                for ages in age_boosting_scenario_order[s]:
                    if np.sum(max_boosters[ss][R][r][ages]) <= scenario_new_boosters[R][r]:
                        # Add boosted from the S, Sf, Sw1, Sw2 to Sw3
                        new_susceptibles_IC[r, ages, 5] += max_boosters[ss][R][r][ages] - max_boosters_for_R[ss][R][r][ages]
                        # Remove boosted from the S
                        new_susceptibles_IC[r, ages, 0] -= max_boosters_for_S[ss][R][r][ages]
                        # Remove boosted from the Sf
                        new_susceptibles_IC[r, ages, 1] -= max_boosters_for_Sf[ss][R][r][ages]
                        # Remove boosted from the Sw1
                        new_susceptibles_IC[r, ages, 3] -= max_boosters_for_Sw1[ss][R][r][ages]
                        # Remove boosted from the Sw2
                        new_susceptibles_IC[r, ages, 4] -= max_boosters_for_Sw2[ss][R][r][ages]
                        # Remove boosted from the S, Sf, Sw1, Sw2 and R from total boosters for the scenario
                        scenario_new_boosters[R][r] -= np.sum(max_boosters[ss][R][r][ages])
                        # Complete boosting vaccination vector
                        boosted[r, ages] = max_boosters[ss][R][r][ages]
                    else:
                        # Compute proportion of boosters we have left to give for the age group
                        prop = scenario_new_boosters[R][r] / np.sum(max_boosters[ss][R][r][ages])
                        # Add boosted from the S, Sf, Sw1, Sw2 to Sw3
                        new_susceptibles_IC[r, ages, 5] += prop * (max_boosters[ss][R][r][ages] - max_boosters_for_R[ss][R][r][ages])
                        # Remove boosted from the S
                        new_susceptibles_IC[r, ages, 0] -= prop * max_boosters_for_S[ss][R][r][ages]
                        # Remove boosted from the Sf
                        new_susceptibles_IC[r, ages, 1] -= prop * max_boosters_for_Sf[ss][R][r][ages]
                        # Remove boosted from the Sw1
                        new_susceptibles_IC[r, ages, 3] -= prop * max_boosters_for_Sw1[ss][R][r][ages]
                        # Remove boosted from the Sw2
                        new_susceptibles_IC[r, ages, 4] -= prop * max_boosters_for_Sw2[ss][R][r][ages]
                        # Remove boosted from the S, Sf, Sw1, Sw2 and R from total boosters for the scenario
                        scenario_new_boosters[R][r] -= prop * np.sum(max_boosters[ss][R][r][ages])
                        # Complete boosting vaccination vector
                        boosted[r, ages] = prop * max_boosters[ss][R][r][ages]

                reg_new_susceptibles_IC.append(list(deepflatten(new_susceptibles_IC[r].transpose())))

            regimes_new_susceptibles_IC.append(reg_new_susceptibles_IC)
            regimes_new_boosted.append(boosted)

        scenario_regimes_new_susceptibles_IC.append(regimes_new_susceptibles_IC)
        scenario_regimes_new_boosted.append(regimes_new_boosted)

    scenario_susceptibles_IC.append(scenario_regimes_new_susceptibles_IC)
    scenario_new_boosted.append(scenario_regimes_new_boosted)

### Set the parameters and initial conditions of the model and bundle everything together

In [18]:
# Instantiate model
model = wm.WarwickLancSEIRModel()

# Set the region names, contact and regional data of the model
model.set_regions(regions)
model.set_age_groups(age_groups)

# List of times at which we wish to evaluate the states of the compartments of the model
times = np.arange(1, total_days+1, 1).tolist()

In [19]:
model.read_contact_data(matrices_contact[0], time_changes_contact[0])
model.read_regional_data(matrices_region[0], time_changes_region[0])

# Set regional and time dependent parameters
regional_parameters = wm.RegParameters(
    model=model,
    region_index=1
)

# Set ICs parameters
ICs_parameters = wm.ICs(
    model=model,
    susceptibles_IC=scenario_susceptibles_IC[0][0][0],
    exposed1_IC=exposed1_IC[0],
    exposed2_IC=exposed2_IC[0],
    exposed3_IC=exposed3_IC[0],
    exposed4_IC=exposed4_IC[0],
    exposed5_IC=exposed5_IC[0],
    infectives_sym_IC=infectives_sym_IC[0],
    infectives_asym_IC=infectives_asym_IC[0],
    recovered_IC=ss_recovered_IC[0][0]
)

# Set disease-specific parameters
disease_parameters = wm.DiseaseParameters(
    model=model,
    d=d[0][0],
    tau=tau[0],
    we=we[0],
    omega=omega[0]
)

# Set transmission parameters
transmission_parameters = wm.Transmission(
    model=model,
    beta=beta[0][0],
    alpha=alpha[0],
    gamma=gamma[0]
)

# Set other simulation parameters
simulation_parameters = wm.SimParameters(
    model=model,
    method='Radau',
    times=times,
    eps=False
)

# Set vaccination parameters
vaccine_parameters = wm.VaccineParameters(
    model=model,
    vac=vac,
    vacb=vacb,
    adult=adult,
    nu_tra=nu_tra[0],
    nu_symp=nu_symp[0],
    nu_inf=nu_inf[0],
    nu_sev_h=nu_sev_h[0],
    nu_sev_d=nu_sev_d[0],
)

# Set social distancing parameters
soc_dist_parameters = wm.SocDistParameters(
    model=model,
    phi=1
)

# Set all parameters in the controller
parameters = wm.ParametersController(
    model=model,
    regional_parameters=regional_parameters,
    ICs_parameters=ICs_parameters,
    disease_parameters=disease_parameters,
    transmission_parameters=transmission_parameters,
    simulation_parameters=simulation_parameters,
    vaccine_parameters=vaccine_parameters,
    soc_dist_parameters=soc_dist_parameters
)

model.simulate(parameters)

array([[    0.        ,     0.        ,     0.        , ...,
            0.        ,     0.        ,     0.        ],
       [  164.37524045,   172.52751241,   165.54876075, ...,
        11300.51980512, 11320.77490579, 19684.11499238],
       [  651.57287528,   683.76927324,   655.18823655, ...,
        11095.31017938, 11036.08273929, 19070.77171956],
       ...,
       [41332.17464569, 40102.07541195, 18765.55467933, ...,
          265.17830154,   197.47001708,  -225.90514307],
       [41654.4322948 , 40452.38052127, 18892.5952608 , ...,
          304.3482779 ,   219.3837423 ,  -173.40885663],
       [41968.3831234 , 40794.35083028, 19013.28887332, ...,
          340.70106627,   239.62547941,  -122.94237156]])

In [20]:
# Incorporate overall reductions in vaccine efficacies
change_vaccine = 0.2

cv_nu_tra = []
cv_nu_symp = []
cv_nu_inf = []
cv_nu_sev_h = []
cv_nu_sev_d = []

for R, regime in enumerate(regimes):
    eff_df = pd.read_csv(
            os.path.join(path, 'global_parameters/efficacies_{}.csv'.format(regime)),
            usecols=range(1,5), dtype=np.float64)
    
    VE_i = np.multiply([1] + [1-change_vaccine] * 4 + [1], eff_df['Infection_eff'])
    VE_s = np.multiply([1] + [1-change_vaccine] * 4 + [1], eff_df['Symptom_eff'])
    VE_h = np.multiply([1] + [1-change_vaccine] * 4 + [1], eff_df['Hosp_eff'])
    VE_d = np.multiply([1] + [1-change_vaccine] * 4 + [1], eff_df['Death_eff'])

    VE_d = np.divide(VE_d-VE_h, 1-VE_h)
    VE_h = np.divide(VE_h-VE_i, 1-VE_i)
    VE_s = np.divide(VE_s-VE_i, 1-VE_i)

    cv_nu_tra.append([1] * 6)
    cv_nu_symp.append(np.nan_to_num(1 - VE_s).tolist()[:-1] + [nu_symp[R][2]])
    cv_nu_inf.append(np.nan_to_num(1 - VE_i).tolist()[:-1] + [nu_inf[R][2]])
    cv_nu_sev_h.append(np.nan_to_num(1 - VE_h).tolist()[:-1] + [nu_sev_h[R][2]])
    cv_nu_sev_d.append(np.nan_to_num(1 - VE_d).tolist()[:-1] + [nu_sev_d[R][2]])

### Compute new boosters available based on each boosting scenario

In [21]:
# Delay in number of days until distributing variant-adapted vaccines 
delay_times = [10, 25, 50, 65, 80, 100, 120, 150]

no_vaccine_outputs = np.load('outputs/overall_vaccine_effects_output.npy')[1, -1, :, :, :, :]

In [22]:
# Compute the maximum number of boosters we can deploy for each age group in each region
delayed_max_boosters = []
delayed_max_boosters_for_R = []
delayed_max_boosters_for_S = []
delayed_max_boosters_for_Sf = []
delayed_max_boosters_for_Sw1 = []
delayed_max_boosters_for_Sw2 = []
delayed_max_boosters_for_Sw3 = []

for dt, delay_time in enumerate(delay_times):
    dt_delayed_max_boosters = []
    dt_delayed_max_boosters_for_R = []
    dt_delayed_max_boosters_for_S = []
    dt_delayed_max_boosters_for_Sf = []
    dt_delayed_max_boosters_for_Sw1 = []
    dt_delayed_max_boosters_for_Sw2 = []
    dt_delayed_max_boosters_for_Sw3 = []

    for R, _ in enumerate(regimes):
        regimes_delayed_max_boosters = []
        regimes_delayed_max_boosters_for_R = []
        regimes_delayed_max_boosters_for_S = []
        regimes_delayed_max_boosters_for_Sf = []
        regimes_delayed_max_boosters_for_Sw1 = []
        regimes_delayed_max_boosters_for_Sw2 = []
        regimes_delayed_max_boosters_for_Sw3 = []

        for r, reg in enumerate(regions):
                delayed_boosters_for_R = boost_age_percent * np.asarray(
                    no_vaccine_outputs[R, r, delay_time-1, (48 * len(age_groups)):(49 * len(age_groups))])
                delayed_boosters_for_S = boost_age_percent * np.asarray(
                    no_vaccine_outputs[R, r, delay_time-1, :len(age_groups)])
                delayed_boosters_for_Sf = boost_age_percent * np.asarray(
                    no_vaccine_outputs[R, r, delay_time-1, len(age_groups):(2*len(age_groups))])
                delayed_boosters_for_Sw1 = boost_age_percent * np.asarray(
                    no_vaccine_outputs[R, r, delay_time-1, (3*len(age_groups)):(4*len(age_groups))])
                delayed_boosters_for_Sw2 = boost_age_percent * np.asarray(
                    no_vaccine_outputs[R, r, delay_time-1, (4*len(age_groups)):(5*len(age_groups))])
                delayed_boosters_for_Sw3 = boost_age_percent * np.asarray(
                    no_vaccine_outputs[R, r, delay_time-1, (5*len(age_groups)):(6*len(age_groups))])

                delayed_boosted = np.asarray(
                    no_vaccine_outputs[R, r, delay_time-1, (2*len(age_groups)):(3*len(age_groups))])

                regimes_delayed_max_boosters_for_R.append(delayed_boosters_for_R)
                regimes_delayed_max_boosters_for_S.append(delayed_boosters_for_S)
                regimes_delayed_max_boosters_for_Sf.append(delayed_boosters_for_Sf)
                regimes_delayed_max_boosters_for_Sw1.append(delayed_boosters_for_Sw1)
                regimes_delayed_max_boosters_for_Sw2.append(delayed_boosters_for_Sw2)
                regimes_delayed_max_boosters_for_Sw3.append(delayed_boosters_for_Sw3)

                regimes_delayed_max_boosters.append(
                        delayed_boosters_for_R + delayed_boosters_for_S + delayed_boosters_for_Sf +
                        delayed_boosters_for_Sw1 + delayed_boosters_for_Sw2 + delayed_boosters_for_Sw3)

        dt_delayed_max_boosters_for_R.append(regimes_delayed_max_boosters_for_R)
        dt_delayed_max_boosters_for_S.append(regimes_delayed_max_boosters_for_S)
        dt_delayed_max_boosters_for_Sf.append(regimes_delayed_max_boosters_for_Sf)
        dt_delayed_max_boosters_for_Sw1.append(regimes_delayed_max_boosters_for_Sw1)
        dt_delayed_max_boosters_for_Sw2.append(regimes_delayed_max_boosters_for_Sw2)
        dt_delayed_max_boosters_for_Sw3.append(regimes_delayed_max_boosters_for_Sw3)
        dt_delayed_max_boosters.append(regimes_delayed_max_boosters)

    delayed_max_boosters_for_R.append(dt_delayed_max_boosters_for_R)
    delayed_max_boosters_for_S.append(dt_delayed_max_boosters_for_S)
    delayed_max_boosters_for_Sf.append(dt_delayed_max_boosters_for_Sf)
    delayed_max_boosters_for_Sw1.append(dt_delayed_max_boosters_for_Sw1)
    delayed_max_boosters_for_Sw2.append(dt_delayed_max_boosters_for_Sw2)
    delayed_max_boosters_for_Sw3.append(dt_delayed_max_boosters_for_Sw3)
    delayed_max_boosters.append(dt_delayed_max_boosters)

# Create list of new susceptible_ICs for each vaccination scenario
scenario_delayed_susceptibles_IC = []
scenario_delayed_new_boosted = []

### Compute new susceptible conditions based on each boosting scenario

In [23]:
# Compute the number of boosters for each region
delayed_total_pop = []

for R in regimes:
        regimes_total_pop = []
        for r in regions:
                IC_df = pd.read_csv(
                        os.path.join(path, '{}/Start_pop_{}.csv'.format(r, R)),
                        header=None, dtype=np.float64)

                regimes_total_pop.append(np.sum(np.asarray(IC_df)))
        delayed_total_pop.append(regimes_total_pop)

In [24]:
for dt, delay_time in enumerate(delay_times):
    scenario_delayed_regimes_new_susceptibles_IC = []
    scenario_delayed_regimes_new_boosted = []
    for s, _ in enumerate(scenario_names):
        delayed_regimes_new_susceptibles_IC = []
        delayed_regimes_new_boosted = []
        scenario_delayed_new_boosters = (scenario_boost_pop_percent[s] * np.array(delayed_total_pop)).tolist()

        for R, _ in enumerate(regimes):
            new_susceptibles_IC = [
                np.array(no_vaccine_outputs[R, r, delay_time-1, :(6 * len(age_groups))].copy()).reshape((6, 16)).transpose() for r in range(len(regions))]
            
            new_susceptibles_IC = np.array(new_susceptibles_IC)
            regimes_scenario_new_boosters = scenario_delayed_new_boosters[R]
            reg_new_susceptibles_IC = []

            boosted = np.zeros_like(new_susceptibles_IC[:, :, 2])

            for r, reg in enumerate(regions):
                for ages in age_boosting_scenario_order[s]:
                    if np.sum(delayed_max_boosters[dt][R][r][ages]) <= scenario_delayed_new_boosters[R][r]:
                        # Add boosted from the S, Sf, Sw1, Sw2, Sw3 to Sb
                        new_susceptibles_IC[r, ages, 2] += delayed_max_boosters[dt][R][r][ages] - delayed_max_boosters_for_R[dt][R][r][ages]
                        # Remove boosted from the S
                        new_susceptibles_IC[r, ages, 0] -= delayed_max_boosters_for_S[dt][R][r][ages]
                        # Remove boosted from the Sf
                        new_susceptibles_IC[r, ages, 1] -= delayed_max_boosters_for_Sf[dt][R][r][ages]
                        # Remove boosted from the Sw1
                        new_susceptibles_IC[r, ages, 3] -= delayed_max_boosters_for_Sw1[dt][R][r][ages]
                        # Remove boosted from the Sw2
                        new_susceptibles_IC[r, ages, 4] -= delayed_max_boosters_for_Sw2[dt][R][r][ages]
                        # Remove boosted from the Sw3
                        new_susceptibles_IC[r, ages, 5] -= delayed_max_boosters_for_Sw3[dt][R][r][ages]
                        # Remove boosted from the S, Sf, Sw1, Sw2, Sw3 and R from total boosters for the scenario
                        scenario_delayed_new_boosters[R][r] -= np.sum(delayed_max_boosters[dt][R][r][ages])
                        # Complete boosting vaccination vector
                        boosted[r, ages] = delayed_max_boosters[dt][R][r][ages]
                    else:
                        # Compute proportion of boosters we have left to give for the age group
                        prop = scenario_delayed_new_boosters[R][r] / np.sum(delayed_max_boosters[dt][R][r][ages])
                        # Add boosted from the S, Sf, Sw1, Sw2, Sw3 to Sb
                        new_susceptibles_IC[r, ages, 2] += prop * (delayed_max_boosters[dt][R][r][ages] - delayed_max_boosters_for_R[dt][R][r][ages])
                        # Remove boosted from the S
                        new_susceptibles_IC[r, ages, 0] -= prop * delayed_max_boosters_for_S[dt][R][r][ages]
                        # Remove boosted from the Sf
                        new_susceptibles_IC[r, ages, 1] -= prop * delayed_max_boosters_for_Sf[dt][R][r][ages]
                        # Remove boosted from the Sw1
                        new_susceptibles_IC[r, ages, 3] -= prop * delayed_max_boosters_for_Sw1[dt][R][r][ages]
                        # Remove boosted from the Sw2
                        new_susceptibles_IC[r, ages, 4] -= prop * delayed_max_boosters_for_Sw2[dt][R][r][ages]
                        # Remove boosted from the Sw3
                        new_susceptibles_IC[r, ages, 5] -= prop * delayed_max_boosters_for_Sw3[dt][R][r][ages]
                        # Remove boosted from the S, Sf, Sw1, Sw2, Sw3 and R from total boosters for the scenario
                        scenario_delayed_new_boosters[R][r] -= prop * np.sum(delayed_max_boosters[dt][R][r][ages])
                        # Complete boosting vaccination vector
                        boosted[r, ages] = prop * delayed_max_boosters[dt][R][r][ages]

                reg_new_susceptibles_IC.append(list(deepflatten(new_susceptibles_IC[r].transpose())))

            delayed_regimes_new_susceptibles_IC.append(reg_new_susceptibles_IC)
            delayed_regimes_new_boosted.append(boosted)

        scenario_delayed_regimes_new_susceptibles_IC.append(delayed_regimes_new_susceptibles_IC)
        scenario_delayed_regimes_new_boosted.append(delayed_regimes_new_boosted)

    scenario_delayed_susceptibles_IC.append(scenario_delayed_regimes_new_susceptibles_IC)
    scenario_delayed_new_boosted.append(scenario_delayed_regimes_new_boosted)

### Simulate for the regions

### Variant-adapted vaccine Delayed Distribution
First half

In [52]:
# Simulate for all the regions and regimes
var_adapted_delayed_outputs1 = []
var_adapted_delayed_total_pop1 = []

for dt, delay_time in enumerate(delay_times[:4]):
    dt_delayed_scenario_outputs = []
    dt_delayed_scenario_total_pop = []
    for s, scenario in enumerate(scenario_names):
        scenario_delayed_outputs = []
        scenario_delayed_total_pop = []
        for R, regime in enumerate(regimes):
            regimes_delayed_outputs = []
            regimes_delayed_total_pop = []
            for r, reg in enumerate(regions):
                model._times = np.arange(1, total_days - delay_time + 1, 1)
                pre_vaccination_outputs = no_vaccine_outputs[R, :, :(delay_time-1), :]

                model.read_contact_data(matrices_contact[R], time_changes_contact[R])
                model.read_regional_data(matrices_region[R], time_changes_region[R])

                # Set regional and time dependent parameters
                regional_parameters = wm.RegParameters(
                    model=model,
                    region_index=1+r
                )

                # Set ICs parameters
                ICs_parameters = wm.ICs(
                    model=model,
                    susceptibles_IC=scenario_delayed_susceptibles_IC[dt][s][R],
                    exposed1_IC=np.hstack((
                        no_vaccine_outputs[R, :, delay_time-1, (6 * len(age_groups)):(7 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (11 * len(age_groups)):(12 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (16 * len(age_groups)):(17 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (21 * len(age_groups)):(22 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (26 * len(age_groups)):(27 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (31 * len(age_groups)):(32 * len(age_groups))]
                    )),
                    exposed2_IC=np.hstack((
                        no_vaccine_outputs[R, :, delay_time-1, (7 * len(age_groups)):(8 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (12 * len(age_groups)):(13 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (17 * len(age_groups)):(18 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (22 * len(age_groups)):(23 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (27 * len(age_groups)):(28 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (32 * len(age_groups)):(33 * len(age_groups))]
                    )),
                    exposed3_IC=np.hstack((
                        no_vaccine_outputs[R, :, delay_time-1, (8 * len(age_groups)):(9 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (13 * len(age_groups)):(14 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (18 * len(age_groups)):(19 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (23 * len(age_groups)):(24 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (28 * len(age_groups)):(29 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (33 * len(age_groups)):(34 * len(age_groups))]
                    )),
                    exposed4_IC=np.hstack((
                        no_vaccine_outputs[R, :, delay_time-1, (9 * len(age_groups)):(10 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (14 * len(age_groups)):(15 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (19 * len(age_groups)):(20 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (24 * len(age_groups)):(25 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (29 * len(age_groups)):(30 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (34 * len(age_groups)):(35 * len(age_groups))]
                    )),
                    exposed5_IC=np.hstack((
                        no_vaccine_outputs[R, :, delay_time-1, (10 * len(age_groups)):(11 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (15 * len(age_groups)):(16 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (20 * len(age_groups)):(21 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (25 * len(age_groups)):(26 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (30 * len(age_groups)):(31 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (35 * len(age_groups)):(36 * len(age_groups))]
                    )),
                    infectives_sym_IC=no_vaccine_outputs[R, :, delay_time-1, (36 * len(age_groups)):(42 * len(age_groups))],
                    infectives_asym_IC=no_vaccine_outputs[R, :, delay_time-1, (42 * len(age_groups)):(48 * len(age_groups))],
                    recovered_IC=no_vaccine_outputs[R, :, delay_time-1, (48 * len(age_groups)):(49 * len(age_groups))]
                )

                # Compute age-dependent population
                regimes_delayed_total_pop.append(ICs_parameters.total_population()[r])

                # Set disease-specific parameters
                disease_parameters = wm.DiseaseParameters(
                    model=model,
                    d=d[R][r],
                    tau=tau[R],
                    we=we[R],
                    omega=omega[R]
                )

                # Set transmission parameters
                transmission_parameters = wm.Transmission(
                    model=model,
                    beta=beta[R][r],
                    alpha=alpha[R],
                    gamma=gamma[R]
                )

                # Set other simulation parameters
                simulation_parameters = wm.SimParameters(
                    model=model,
                    method='Radau',
                    times=np.arange(1, 2+total_days - delay_time, 1).tolist(),
                    eps=False
                )

                # Set vaccination parameters
                vaccine_parameters = wm.VaccineParameters(
                    model=model,
                    vac=vac,
                    vacb=vacb,
                    adult=adult,
                    nu_tra=nu_tra[R],
                    nu_symp=nu_symp[R],
                    nu_inf=nu_inf[R],
                    nu_sev_h=nu_sev_h[R],
                    nu_sev_d=nu_sev_d[R],
                )

                # Set social distancing parameters
                soc_dist_parameters = wm.SocDistParameters(
                    model=model,
                    phi=1
                )

                # Set all parameters in the controller
                parameters = wm.ParametersController(
                    model=model,
                    regional_parameters=regional_parameters,
                    ICs_parameters=ICs_parameters,
                    disease_parameters=disease_parameters,
                    transmission_parameters=transmission_parameters,
                    simulation_parameters=simulation_parameters,
                    vaccine_parameters=vaccine_parameters,
                    soc_dist_parameters=soc_dist_parameters
                )

                # Simulate using the ODE solver
                if delay_time == total_days:
                    regimes_delayed_outputs.append(no_vaccine_outputs[R, r, :delay_time, :])
                else:
                    regimes_delayed_outputs.append(np.vstack((pre_vaccination_outputs[r, :, :], model.simulate(parameters))))

            scenario_delayed_outputs.append(regimes_delayed_outputs)
            scenario_delayed_total_pop.append(regimes_delayed_total_pop)

        dt_delayed_scenario_outputs.append(scenario_delayed_outputs)
        dt_delayed_scenario_total_pop.append(scenario_delayed_total_pop)

    var_adapted_delayed_outputs1.append(dt_delayed_scenario_outputs)
    var_adapted_delayed_total_pop1.append(dt_delayed_scenario_total_pop)

var_adapted_delayed_outputs1 = np.array(var_adapted_delayed_outputs1)

Second half

In [53]:
# Simulate for all the regions and regimes
var_adapted_delayed_outputs2 = []
var_adapted_delayed_total_pop2 = []

for dt, delay_time in enumerate(delay_times[4:]):
    dt_delayed_scenario_outputs = []
    dt_delayed_scenario_total_pop = []
    for s, scenario in enumerate(scenario_names):
        scenario_delayed_outputs = []
        scenario_delayed_total_pop = []
        for R, regime in enumerate(regimes):
            regimes_delayed_outputs = []
            regimes_delayed_total_pop = []
            for r, reg in enumerate(regions):
                model._times = np.arange(1, total_days - delay_time + 1, 1)
                pre_vaccination_outputs = no_vaccine_outputs[R, :, :(delay_time-1), :]

                model.read_contact_data(matrices_contact[R], time_changes_contact[R])
                model.read_regional_data(matrices_region[R], time_changes_region[R])

                # Set regional and time dependent parameters
                regional_parameters = wm.RegParameters(
                    model=model,
                    region_index=1+r
                )

                # Set ICs parameters
                ICs_parameters = wm.ICs(
                    model=model,
                    susceptibles_IC=scenario_delayed_susceptibles_IC[dt+4][s][R],
                    exposed1_IC=np.hstack((
                        no_vaccine_outputs[R, :, delay_time-1, (6 * len(age_groups)):(7 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (11 * len(age_groups)):(12 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (16 * len(age_groups)):(17 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (21 * len(age_groups)):(22 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (26 * len(age_groups)):(27 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (31 * len(age_groups)):(32 * len(age_groups))]
                    )),
                    exposed2_IC=np.hstack((
                        no_vaccine_outputs[R, :, delay_time-1, (7 * len(age_groups)):(8 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (12 * len(age_groups)):(13 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (17 * len(age_groups)):(18 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (22 * len(age_groups)):(23 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (27 * len(age_groups)):(28 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (32 * len(age_groups)):(33 * len(age_groups))]
                    )),
                    exposed3_IC=np.hstack((
                        no_vaccine_outputs[R, :, delay_time-1, (8 * len(age_groups)):(9 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (13 * len(age_groups)):(14 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (18 * len(age_groups)):(19 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (23 * len(age_groups)):(24 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (28 * len(age_groups)):(29 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (33 * len(age_groups)):(34 * len(age_groups))]
                    )),
                    exposed4_IC=np.hstack((
                        no_vaccine_outputs[R, :, delay_time-1, (9 * len(age_groups)):(10 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (14 * len(age_groups)):(15 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (19 * len(age_groups)):(20 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (24 * len(age_groups)):(25 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (29 * len(age_groups)):(30 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (34 * len(age_groups)):(35 * len(age_groups))]
                    )),
                    exposed5_IC=np.hstack((
                        no_vaccine_outputs[R, :, delay_time-1, (10 * len(age_groups)):(11 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (15 * len(age_groups)):(16 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (20 * len(age_groups)):(21 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (25 * len(age_groups)):(26 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (30 * len(age_groups)):(31 * len(age_groups))],
                        no_vaccine_outputs[R, :, delay_time-1, (35 * len(age_groups)):(36 * len(age_groups))]
                    )),
                    infectives_sym_IC=no_vaccine_outputs[R, :, delay_time-1, (36 * len(age_groups)):(42 * len(age_groups))],
                    infectives_asym_IC=no_vaccine_outputs[R, :, delay_time-1, (42 * len(age_groups)):(48 * len(age_groups))],
                    recovered_IC=no_vaccine_outputs[R, :, delay_time-1, (48 * len(age_groups)):(49 * len(age_groups))]
                )

                # Compute age-dependent population
                regimes_delayed_total_pop.append(ICs_parameters.total_population()[r])

                # Set disease-specific parameters
                disease_parameters = wm.DiseaseParameters(
                    model=model,
                    d=d[R][r],
                    tau=tau[R],
                    we=we[R],
                    omega=omega[R]
                )

                # Set transmission parameters
                transmission_parameters = wm.Transmission(
                    model=model,
                    beta=beta[R][r],
                    alpha=alpha[R],
                    gamma=gamma[R]
                )

                # Set other simulation parameters
                simulation_parameters = wm.SimParameters(
                    model=model,
                    method='Radau',
                    times=np.arange(1, 2+total_days - delay_time, 1).tolist(),
                    eps=False
                )

                # Set vaccination parameters
                vaccine_parameters = wm.VaccineParameters(
                    model=model,
                    vac=vac,
                    vacb=vacb,
                    adult=adult,
                    nu_tra=nu_tra[R],
                    nu_symp=nu_symp[R],
                    nu_inf=nu_inf[R],
                    nu_sev_h=nu_sev_h[R],
                    nu_sev_d=nu_sev_d[R],
                )

                # Set social distancing parameters
                soc_dist_parameters = wm.SocDistParameters(
                    model=model,
                    phi=1
                )

                # Set all parameters in the controller
                parameters = wm.ParametersController(
                    model=model,
                    regional_parameters=regional_parameters,
                    ICs_parameters=ICs_parameters,
                    disease_parameters=disease_parameters,
                    transmission_parameters=transmission_parameters,
                    simulation_parameters=simulation_parameters,
                    vaccine_parameters=vaccine_parameters,
                    soc_dist_parameters=soc_dist_parameters
                )

                # Simulate using the ODE solver
                if delay_time == total_days:
                    regimes_delayed_outputs.append(no_vaccine_outputs[R, r, :delay_time, :])
                else:
                    regimes_delayed_outputs.append(np.vstack((pre_vaccination_outputs[r, :, :], model.simulate(parameters))))

            scenario_delayed_outputs.append(regimes_delayed_outputs)
            scenario_delayed_total_pop.append(regimes_delayed_total_pop)

        dt_delayed_scenario_outputs.append(scenario_delayed_outputs)
        dt_delayed_scenario_total_pop.append(scenario_delayed_total_pop)

    var_adapted_delayed_outputs2.append(dt_delayed_scenario_outputs)
    var_adapted_delayed_total_pop2.append(dt_delayed_scenario_total_pop)

var_adapted_delayed_outputs2 = np.array(var_adapted_delayed_outputs2)

## Save the model output data

In [54]:
# Save output as a numpy matrix file
np.save('outputs/variant_adapted_vaccine_delayed_outputs1', var_adapted_delayed_outputs1)
np.save('outputs/variant_adapted_vaccine_delayed_pop1', var_adapted_delayed_total_pop1)

np.save('outputs/variant_adapted_vaccine_delayed_outputs2', var_adapted_delayed_outputs2)
np.save('outputs/variant_adapted_vaccine_delayed_pop2', var_adapted_delayed_total_pop2)

## Reload model output data

In [25]:
current_vacc_outputs = np.load('outputs/overall_vaccine_effects_output.npy')[1, :, :, :, :, :]
current_vacc_total_pop = np.load('outputs/overall_vaccine_effects_pop.npy').tolist()[1]

var_adapted_delayed_outputs1 = np.load('outputs/variant_adapted_vaccine_delayed_outputs1.npy')
var_adapted_delayed_total_pop1 = np.load('outputs/variant_adapted_vaccine_delayed_pop1.npy')

var_adapted_delayed_outputs2 = np.load('outputs/variant_adapted_vaccine_delayed_outputs2.npy')
var_adapted_delayed_total_pop2 = np.load('outputs/variant_adapted_vaccine_delayed_pop2.npy')

var_adapted_delayed_outputs = np.concatenate((var_adapted_delayed_outputs1, var_adapted_delayed_outputs2), axis=0)
var_adapted_delayed_total_pop = np.concatenate((var_adapted_delayed_total_pop1, var_adapted_delayed_total_pop2), axis=0).tolist()

## Plot the comparments of the two methods against each other
### Setup ``plotly`` and default settings for plotting

In [26]:
from plotly.subplots import make_subplots

colours = ['blue', 'orange', 'green', 'red', 'purple', 'deepskyblue', 'magenta', 'grey']

## Number of New Deaths Current Vaccine

In [33]:
# Simulate for all the regions
model._times = np.arange(1, 1+total_days, 1)

current_vacc_total_new_deaths = []

for s, scenario in enumerate(scenario_names):
    scenario_total_new_deaths = []
    for R, regime in enumerate(regimes):
        regimes_total_new_deaths = []

        model.vaccine_param[6] = cv_nu_sev_h[R]
        model.vaccine_param[7] = cv_nu_sev_d[R]
        for r, reg in enumerate(regions):
            # Compute regional matrix of new symptomatic infections for all timepoints simulated
            reg_new_infections = model.new_infections(current_vacc_outputs[s, R, r, :, :])

            # Compute regional matrix of new hospitalisation for all timepoints simulated
            reg_new_hospitalisation = model.new_hospitalisations(reg_new_infections, pItoH[R][r], dItoH[R])

            # Compute regional matrix of new deaths for all timepoints simulated
            reg_new_deaths = model.new_deaths(reg_new_hospitalisation, pHtoD[R][r], dHtoD[R])
            
            regimes_total_new_deaths.append(((10**5) / np.sum(current_vacc_total_pop[s][R][r])) * np.sum( 
                reg_new_deaths[0] + reg_new_deaths[1] + reg_new_deaths[2] +
                reg_new_deaths[3] + reg_new_deaths[4] + reg_new_deaths[5],
                axis=1))

        scenario_total_new_deaths.append(regimes_total_new_deaths)

    current_vacc_total_new_deaths.append(scenario_total_new_deaths)

current_vacc_total_new_deaths = np.array(current_vacc_total_new_deaths)

## Number of New Deaths Delayed Variant-adapted Vaccine

In [41]:
# Simulate for all the regions
var_adapted_delayed_total_new_deaths = []

for dt, delay_time in enumerate(delay_times):
    dt_total_new_deaths = []
    for s, scenario in enumerate(scenario_names):
        scenario_total_new_deaths = []
        for R, regime in enumerate(regimes):
            regimes_total_new_deaths = []
            pre_delay_regimes_total_new_deaths = []
            post_delay_regimes_total_new_deaths = []

            model._times = np.arange(1, delay_time+1, 1)
            model.vaccine_param[6] = cv_nu_sev_h[R]
            model.vaccine_param[7] = cv_nu_sev_d[R]
            for r, reg in enumerate(regions):
                # Compute regional matrix of new symptomatic infections for all timepoints simulated
                reg_new_infections = model.new_infections(var_adapted_delayed_outputs[dt, s, R, r, :delay_time, :])

                # Compute regional matrix of new hospitalisation for all timepoints simulated
                reg_new_hospitalisation = model.new_hospitalisations(reg_new_infections, pItoH[R][r], dItoH[R])

                # Compute regional matrix of new deaths for all timepoints simulated
                reg_new_deaths = model.new_deaths(reg_new_hospitalisation, pHtoD[R][r], dHtoD[R])
                
                pre_delay_regimes_total_new_deaths.append(((10**5) / np.sum(var_adapted_delayed_total_pop[dt][s][R][r])) * np.sum( 
                    reg_new_deaths[0] + reg_new_deaths[1] + reg_new_deaths[2] +
                    reg_new_deaths[3] + reg_new_deaths[4] + reg_new_deaths[5],
                    axis=1))

            model._times = np.arange(1, total_days - delay_time + 1, 1)
            model.vaccine_param[6] = nu_sev_h[R]
            model.vaccine_param[7] = nu_sev_d[R]
            for r, reg in enumerate(regions):
                # Compute regional matrix of new symptomatic infections for all timepoints simulated
                reg_new_infections = model.new_infections(var_adapted_delayed_outputs[dt, s, R, r, delay_time:, :])

                # Compute regional matrix of new hospitalisation for all timepoints simulated
                reg_new_hospitalisation = model.new_hospitalisations(reg_new_infections, pItoH[R][r], dItoH[R])

                # Compute regional matrix of new deaths for all timepoints simulated
                reg_new_deaths = model.new_deaths(reg_new_hospitalisation, pHtoD[R][r], dHtoD[R])
                
                post_delay_regimes_total_new_deaths.append(((10**5) / np.sum(var_adapted_delayed_total_pop[dt][s][R][r])) * np.sum( 
                    reg_new_deaths[0] + reg_new_deaths[1] + reg_new_deaths[2] +
                    reg_new_deaths[3] + reg_new_deaths[4] + reg_new_deaths[5],
                    axis=1))

            for r, reg in enumerate(regions):
                regimes_total_new_deaths.append(np.concatenate(
                    (pre_delay_regimes_total_new_deaths[r], post_delay_regimes_total_new_deaths[r])))

            scenario_total_new_deaths.append(regimes_total_new_deaths)
        
        dt_total_new_deaths.append(scenario_total_new_deaths)

    var_adapted_delayed_total_new_deaths.append(dt_total_new_deaths)

var_adapted_delayed_total_new_deaths = np.array(var_adapted_delayed_total_new_deaths)

In [42]:
# Set up traces to plot
current_vacc_total_new_deaths_mean = []
current_vacc_total_new_deaths_upper = []
current_vacc_total_new_deaths_lower = []

var_adapted_delayed_total_new_deaths_mean = []
var_adapted_delayed_total_new_deaths_upper = []
var_adapted_delayed_total_new_deaths_lower = []

for r, _ in enumerate(regions):
    # Compute the mean
    current_vacc_total_new_deaths_mean.append(np.mean(current_vacc_total_new_deaths[:,:,r,:], axis=1))
    var_adapted_delayed_total_new_deaths_mean.append(np.mean(var_adapted_delayed_total_new_deaths[:,:,:,r,:], axis=2))
    
    # Compute the upper quantiles
    current_vacc_total_new_deaths_upper.append(np.quantile(current_vacc_total_new_deaths[:,:,r,:], 0.975, axis=1))
    var_adapted_delayed_total_new_deaths_upper.append(np.quantile(var_adapted_delayed_total_new_deaths[:,:,:,r,:], 0.975, axis=2))
    
    # Compute the lower quantiles
    current_vacc_total_new_deaths_lower.append(np.quantile(current_vacc_total_new_deaths[:,:,r,:], 0.025, axis=1))    
    var_adapted_delayed_total_new_deaths_lower.append(np.quantile(var_adapted_delayed_total_new_deaths[:,:,:,r,:], 0.025, axis=2))

## Summary Figures
### Time of switch from current to variant adapted vaccines

In [43]:
new_scenario_names = ['Scenario {}'.format(s+1) for s, scenario in enumerate(scenario_names[:-1])] + [scenario_names[-1]]

# Plot for each region
for s, scenario in enumerate(new_scenario_names):
    # Plot for each boosting scenario
    for r, reg in enumerate(regions):
        fig = go.Figure()
        fig.add_trace(
            go.Scatter(
                y=[np.sum(var_adapted_delayed_total_new_deaths_mean[r][dt, s, :]) for dt, delay_time in enumerate(delay_times)],
                x=delay_times,
                mode='lines',
                name='Variant-adapted vaccine with delay',
                line=dict(
                    color=colours[r]),
            )
        )

        fig.add_trace(
            go.Scatter(
                y=[np.sum(var_adapted_delayed_total_new_deaths_upper[r][dt, s, :]) for dt, delay_time in enumerate(delay_times)] + [np.sum(var_adapted_delayed_total_new_deaths_lower[r][dt, s, :]) for dt, delay_time in enumerate(delay_times)][::-1],
                x=delay_times + delay_times[::-1],
                mode='lines',
                name='Variant-adapted vaccine with delay',
                fill='toself',
                fillcolor=colours[r],
                line_color=colours[r],
                opacity=0.15,
                showlegend=False
            )
        )

        fig.add_trace(
            go.Scatter(
                y=[np.sum(current_vacc_total_new_deaths_mean[r][s,:])] * len(delay_times),
                x=delay_times,
                mode='lines',
                name='Current vaccine without delay',
                line=dict(
                    color='black',
                    dash='dash')
            )
        )

        # Add axis labels
        fig.update_layout(
            boxmode='group',
            title='Total deaths per 100,000 current vaccine vs variant adapted vaccines for <br>{} {}'.format(reg, scenario),
            width=1100,
            height=600,
            plot_bgcolor='white',
            xaxis=dict(
                linecolor='black',
                title='Delay Time (days)'
                ),
            yaxis=dict(
                linecolor='black',
                title='Total Deaths in 150 days (per 100k)',
                range=[0, 8]),
            hovermode='x unified'
            )

        fig.write_image('images/report5_time_switch_vaccine_type/Time of switch from current to variant adapted vaccines {} {}.pdf'.format(reg, scenario))
        fig.show()